In [1]:
# Importing libraries
import numpy as np
import pandas as pd
import os
import shutil

In [3]:
# Importing the cohort
squamous = pd.read_csv('./tissue-slide-csv/cohort_squamous.csv')
adenocarcinoma = pd.read_csv('./tissue-slide-csv/cohort_adenocarcinoma.csv')

# Checking the column names
squamous.head(3)

,_id,Case_ID,Specimen_ID,Slide_ID,Tumor,Topographic_Site,Specimen_Type,Weight,Tumor_Site,Percent_Tumor_Nuclei,Percent_Total_Cellularity,Percent_Necrosis,Volume,Percent_Blast,Genomics,Proteomics,Radiology,Pathology,HasRadiology
0,"{""$oid"":""5e1c91cb2fda72087ae45c51""}",C3L-05428,C3L-05428-07,C3L-05428-27,LSCC,Lung,normal_tissue,426,NaN,-1,-1,-1,-1,-1,NaN,NaN,NaN,C3L-05428-27,no
1,"{""$oid"":""5e1c91cb2fda72087ae45411""}",C3L-02130,C3L-02130-07,C3L-02130-26,LSCC,Lung,normal_tissue,199,NaN,-1,-1,-1,-1,-1,NaN,NaN,NaN,C3L-02130-26,no
2,"{""$oid"":""5e1c91cb2fda72087ae45410""}",C3L-02130,C3L-02130-01,C3L-02130-21,LSCC,Lung,tumor_tissue,905,Left Lower Lobe,65,90,5,-1,-1,NaN,NaN,NaN,C3L-02130-21,no


In [6]:
# Checking the column names
adenocarcinoma.head(3)

,_id,Case_ID,Specimen_ID,Slide_ID,Tumor,Topographic_Site,Specimen_Type,Weight,Tumor_Site,Percent_Tumor_Nuclei,...,Proteomics,Radiology,Pathology,HasRadiology,Age,Ethnicity,Gender,Race,vital_status_at_12months_follow_up,vital_status_at_24months_follow_up
0,"{""$oid"":""5e1c91cb2fda72087ae46408""}",C3N-02718,C3N-02718-02,C3N-02718-22,LUAD,Lung,tumor_tissue,420,Left Upper Lobe,65,...,NaN,NaN,C3N-02718-22,no,60-70,NaN,Male,NaN,Living,NaN
1,"{""$oid"":""5e456fda2fda72087a73c0a1""}",C3L-04743,C3L-04743-02,C3L-04743-22,LUAD,Lung,tumor_tissue,208,Left Upper Lobe,70,...,NaN,NaN,C3L-04743-22,no,NaN,NaN,NaN,NaN,NaN,NaN
2,"{""$oid"":""5e1c91cb2fda72087ae45067""}",C3L-00446,C3L-00446-06,C3L-00446-25,LUAD,Lung,normal_tissue,100,NaN,-1,...,NaN,NaN,C3L-00446-25,no,70-80,NaN,Male,NaN,Deceased,NaN


In [4]:
# Selecting the slide_id and specimen_type to push images into the correct folders
squamous = squamous[['Slide_ID', 'Specimen_Type']]

# Re-checking columns 
squamous.head(3)

,Slide_ID,Specimen_Type
0,C3L-05428-27,normal_tissue
1,C3L-02130-26,normal_tissue
2,C3L-02130-21,tumor_tissue


In [7]:
# Selecting the slide_id and specimen_type to push images into the correct folders
adenocarcinoma = adenocarcinoma[['Slide_ID', 'Specimen_Type']]

# Re-checking columns 
adenocarcinoma.head(3)

,Slide_ID,Specimen_Type
0,C3N-02718-22,tumor_tissue
1,C3L-04743-22,tumor_tissue
2,C3L-00446-25,normal_tissue


In [ ]:
# Seperating the Normal from the cancerous tissue
squamous_cancerous = squamous.loc[squamous['Specimen_Type'] == 'tumor_tissue']
squamous_normal = squamous.loc[squamous['Specimen_Type'] == 'normal_tissue']
adenocarcinoma_cancerous = adenocarcinoma.loc[adenocarcinoma['Specimen_Type'] == 'tumor_tissue']
adenocarcinoma_normal = adenocarcinoma.loc[adenocarcinoma['Specimen_Type'] == 'normal_tissue']

In [8]:
df = pd.concat([adenocarcinoma, squamous])
df.duplicated().sum()
# No duplicates

0

### Training versus Testing
#### Downloaded 100 images from both the squamous and adenocarcinoma resource
#### Setting the split to 80/20

In [ ]:
count_T1_T = 0
count_T1_N = 0
count_T2_T = 0
count_T2_N = 0

def folder_seperator(folder, type1_tumor, type1_normal, 
                             type2_tumor, type2_normal,
                     train_t1_tumor, train_t2_tumor, train_normal,
                     test_t1_tumor, test_t2_tumor, test_normal):
    
    for i, file in enumerate(os.listdir(folder)):
        
        # Push to Training/Squamous Tissue
        global count_T1_T
        if type1_tumor['Slide_ID'].str.contains(file.replace('.svs', '')).any() and count_T1_T <= 40):
            count_T1_T += 1
            shutil.move(f'{folder}{file}', f"{train_t1_tumor}{file}")

        # Push to Testing/Squamous Tissue   
        elif type1_tumor['Slide_ID'].str.contains(file.replace('.svs', '')).any():
            shutil.move(f'{folder}{file}', f"{test_t1_tumor}{file}")
    
        # Push to Training/Normal Tissue
            global count_T1_N
        elif type1_normal['Slide_ID'].str.contains(file.replace('.svs', '')).any() and count_T1_N <= 40):
            count_T1_N += 1
            shutil.move(f'{folder}{file}', f"{train_normal}{file}")

        # Push to Testing/Normal Tissue
        elif type1_normal['Slide_ID'].str.contains(file.replace('.svs', '')).any():
            shutil.move(f'{folder}{file}', f"{test_normal}{file}")

        # Push to Training/Adenocarcinoma Tissue
            global count_T2_T
        elif type2_tumor['Slide_ID'].str.contains(file.replace('.svs', '')).any() and count_T2_T <= 40):
            count_T2_T += 1
            shutil.move(f'{folder}{file}', f"{train_t2_tumor}{file}")

        # Push to Testing/Adenocarcinoma Tissue
        elif type2_tumor['Slide_ID'].str.contains(file.replace('.svs', '')).any():
            shutil.move(f'{folder}{file}', f"{test_t2_tumor}{file}")

        # Push to Training/Normal Tissue
            global count_T2_N
        elif type2_normal['Slide_ID'].str.contains(file.replace('.svs', '')).any() and count_T2_N <= 40):
            count_T2_N += 1 
            shutil.move(f'{folder}{file}', f"{train_normal}{file}")

        # Push to Testing/Normal Tissue
        elif type2_normal['Slide_ID'].str.contains(file.replace('.svs', '')).any():
            shutil.move(f'{folder}{file}', f"{test_normal}{file}")

        else:
            pass

In [1]:
# Setting up folder seperation for AWS
folder_seperator(folder = '/Slides/', 
                 type1_tumor = squamous_cancerous, type1_normal = squamous_normal,
                 type2_tumor = adenocarcinoma_cancerous, type2_normal = adenocarcinoma_normal,
        train_t1_tumor='/Train/squamous/', 
        train_t2_tumor='/Train/adenocarcinoma/', 
        train_normal='/Train/normal/',
        test_t1_tumor='/Test/squamous/', 
        test_t2_tumor='/Test/adenocarcinoma/', 
        test_normal='/Test/normal/')